In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import time

import CustomHyperModel
import CustomWindowGenerator
import EnergyPricesLibrary as Ep

from kerastuner.tuners import BayesianOptimization

%load_ext autoreload
%autoreload 2

In [2]:
data_horaria_path = os.path.join('..','..','dataset','Series','Sabanas','Original','Sabana_Datos_Horaria.xlsx')
data_horaria = pd.read_excel(data_horaria_path)
data_horaria = data_horaria.set_index('Fecha')

In [3]:
data_diaria_path = os.path.join('..','..','dataset','Series','Sabanas','Original','Sabana_Datos_Diaria.xlsx')
data_diaria = pd.read_excel(data_diaria_path)
data_diaria = data_diaria.set_index('Fecha')

In [4]:
precio_bolsa_path = os.path.join('..','..','dataset','Series','Sabanas','Original','Sabana_Datos_Precio_Bolsa.xlsx')
precio_bolsa = pd.read_excel(precio_bolsa_path)
precio_bolsa = precio_bolsa.set_index('Fecha')

In [5]:
data_horaria_full = pd.concat([data_horaria,precio_bolsa],axis=1)
nombre_series = data_horaria_full.columns

data_horaria_full.shape,data_horaria.shape,precio_bolsa.shape,data_diaria.shape

((176760, 85), (176760, 84), (176760, 1), (7365, 119))

In [6]:
d = 'All'
IndLastMonth = '2020-01-01'
n_steps_in = 24
n_steps_out=24
overlap = 24
inputs_columns = data_horaria_full.columns
output_columns = ['$kWh']

len_input_features = len(inputs_columns)
len_output_features = len(output_columns)

trainX_H, trainY_H, testX_H, testY_H, scaler_H_x,scaler_H_y, df2_H, dataset_H = Ep.SplitTimeseriesMultipleTimesBackAhead(data_horaria_full,
                                                                                              day = d, 
                                                                                              ValData = 'index', 
                                                                                              TimeAhead = IndLastMonth, 
                                                                                              n_steps_out= n_steps_out, 
                                                                                              n_steps_in = n_steps_in, 
                                                                                              overlap = overlap,
                                                                                              input_features=inputs_columns,
                                                                                              output_features=output_columns)
trainX_H.shape,trainY_H.shape,testX_H.shape,testY_H.shape

((7273, 24, 85), (7273, 24, 1), (91, 24, 85), (91, 24, 1))

In [7]:
d = 'All'
time_split = '2020-01-01'
n_steps_in = 1
n_steps_out=24
overlap = 1
inputs_columns = data_diaria.columns.values
output_columns = ['$kWh']

len_input_features = len(inputs_columns)
len_output_features = len(output_columns)

trainX_D, trainY_D, testX_D, testY_D, scaler_D_x,scaler_D_y, dataset_x_D, dataset_y_D = Ep.SplitTimeseriesMultipleTimesBackAhead_differentTimes(
    df_x=data_diaria,
    df_y=precio_bolsa,
    day = d, 
    TimeSplit = time_split,
    n_steps_out=n_steps_out,
    n_steps_in =n_steps_in, 
    overlap = overlap,
    input_features=inputs_columns,
    output_features=output_columns)

trainX_D.shape,trainY_D.shape,testX_D.shape,testY_D.shape

((7273, 1, 119), (7273, 24, 1), (91, 1, 119), (91, 24, 1))

# AutoCorrelación

In [ ]:
from pandas import Series
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf

fig, ax = plt.subplots(figsize=(15,8))
T = precio_bolsa.iloc[-2184:]
plot_acf(T,lags=T.shape[0]-1,title='Autocorrelación 2020/01/01 - 2020/03/31',ax=ax)
plt.show()

In [ ]:
from statsmodels.graphics.tsaplots import plot_pacf

fig, ax = plt.subplots(figsize=(15,8))
T = precio_bolsa.iloc[-2184:]
plot_pacf(T,lags=(T.shape[0]-1)/10,title='Autocorrelación Parcial 2020/01/01 - 2020/03/31',ax=ax)
plt.show()